In [ ]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
import xlsxwriter
import glob
import os
from datetime import datetime
pd.options.display.float_format = '{:,.2f}'.format

# READING FILES

In [ ]:
date_respuesta = '2022-07-07'
date_respuesta = pd.to_datetime(date_respuesta)

## RESPUESTAS FP SIN CLAVES DE RASTREO

In [ ]:
names_resp = ['report_id','status','report_date_local','amount','disbursement_batch_id','merchant_id','bank_account_number',
             'bank_name','bank_account_owner_name','execution_date','aplication_date','disbursement_tracking_key',
             'error_description','reference_number','cep']

In [ ]:
rs_previo = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2022\07. JULIO\Respuesta-FP*.xlsx', recursive=False)
rs_previo

In [ ]:
rs_concat_previo = pd.concat([pd.read_excel(f,sheet_name='Report',names=names_resp,
                                     converters={'bank_account_number':str,'disbursement_tracking_key':str,'report_id':str})
                       .assign(file_name=os.path.basename(f)) for f in rs_previo])
rs_concat_previo.loc[rs_concat_previo['file_name'].str.contains('-1'), 'Time_rs'] = '09:00'
rs_concat_previo.loc[rs_concat_previo['file_name'].str.contains('-2'), 'Time_rs'] = '11:00'
rs_concat_previo.loc[rs_concat_previo['file_name'].str.contains('-3'), 'Time_rs'] = '14:00'
rs_concat_previo.loc[rs_concat_previo['file_name'].str.contains('-4'), 'Time_rs'] = '16:00'
rs_concat_previo.loc[rs_concat_previo['file_name'].str.contains('-5'), 'Time_rs'] = '18:00'
rs_concat_previo.loc[rs_concat_previo['file_name'].str.contains('-6'), 'Time_rs'] = '20:00'
rs_concat_previo.loc[rs_concat_previo['file_name'].str.contains('CITI'), "Banco_emisor"] = 'BANAMEX'
rs_concat_previo.loc[rs_concat_previo['file_name'].str.contains('BNTE'), "Banco_emisor"] = 'BANORTE'
rs_concat_previo.loc[rs_concat_previo['file_name'].str.contains('SANT'), "Banco_emisor"] = 'SANTANDER'
rs_concat_previo.loc[rs_concat_previo['Banco_emisor'] == 'BANAMEX', "Banco emisor"] = '40002'
rs_concat_previo.loc[rs_concat_previo['Banco_emisor'] == 'BANORTE', "Banco emisor"] = '40072'
rs_concat_previo.loc[rs_concat_previo['Banco_emisor'] == 'SANTANDER', "Banco emisor"] = '40014'
rs_concat_previo['INDEX'] = (rs_concat_previo['report_id'].astype(str).replace(' ', '',regex=True)+'_'
                       +rs_concat_previo['Time_rs'].astype(str).replace(' ', '',regex=True))
rs_concat_previo.sort_values(by=['Time_rs'], ascending=True, inplace=True)
rs_concat_previo.head()

In [ ]:
rs_concat_previo.shape

In [ ]:
rs_concat_previo.columns

In [ ]:
duplicates = rs_concat_previo[rs_concat_previo.duplicated(['report_id'] , keep=False)]
duplicates.head()

In [ ]:
duplicates.shape

In [ ]:
rs_concat_previo.to_excel(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2022\07. JULIO\ARCHIVO RESPUESTAS FP SIN CLAVE RASTREO\Respuesta-FP-previa 2022-07-07.xlsx',index=False)

# RESPUESTA FP COMPLETA

In [ ]:
rs = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2022\07. JULIO\ARCHIVO RESPUESTAS FP SIN CLAVE RASTREO\Respuesta-FP-previa 2022-07-07*.xlsx', recursive=False)
rs

In [ ]:
rs_concat = pd.concat([pd.read_excel(f,sheet_name='Sheet1',
                                     converters={'bank_account_number':str,'disbursement_tracking_key':str,'report_id':str})
                       .assign(file_name=os.path.basename(f)) for f in rs])
rs_concat.head()

In [ ]:
rs_concat.shape

## QUERY PAYMENTS

In [ ]:
query = pd.read_csv(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\07. JULIO\FP-ALL 2022-07-07.csv')
query['batch_created_date'] = pd.to_datetime(query['batch_created_date'])
query.set_index('batch_created_date',inplace=True)
query.head()

In [ ]:
query.shape

In [ ]:
hour1 = query.between_time('08:00','10:59')
hour1['Time'] = '09:00'
hour2 = query.between_time('11:00','13:59')
hour2['Time'] = '11:00'
hour3 = query.between_time('14:00','15:59')
hour3['Time'] = '14:00'
hour4 = query.between_time('16:00','17:59')
hour4['Time'] = '16:00'
hour5 = query.between_time('18:00','19:59')
hour5['Time'] = '18:00'
hour6 = query.between_time('20:00','23:59')
hour6['Time'] = '20:00'

In [ ]:
disp1 = pd.concat([hour1,hour2,hour3,hour4,hour5,hour6])
disp1['INDEX'] = (disp1['report_id'].astype(str).replace(' ', '',regex=True)+'_'
                       +disp1['Time'].astype(str).replace(' ', '',regex=True))
disp1.head()

In [ ]:
disp1.shape

## BIN

In [ ]:
bin1 = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\Catalogo_Bancos.csv'
                  ,dtype={'BIN':str,'Clave de la institución':str,'Nombre de la institución':str})


In [ ]:
bin1.head()

In [ ]:
bin1.shape

# ARCHIVO RESPUESTA

In [ ]:
rs_concat['Date_respuesta'] = date_respuesta
rs_concat.rename(columns ={'bank_account_number':'CLABE'}, inplace=True)
rs_concat['BIN'] = rs_concat['CLABE'].str[:3].replace(' ', '',regex=True)
rs_concat.head()

In [ ]:
rs_concat.shape

In [ ]:
rs_concat_2 = rs_concat.merge(bin1[['BIN','Clave de la institución','Nombre de la institución']] ,on='BIN', how='left')
rs_concat_2['LINEA CEP'] = (rs_concat_2['Date_respuesta'].astype(str).replace(' ', '',regex=True)+','
                            +rs_concat_2['disbursement_tracking_key'].astype(str).replace(' ', '',regex=True)+','
                            +rs_concat_2['Banco emisor'].astype(str).replace(' ', '',regex=True)+','
                            +rs_concat_2['Clave de la institución'].astype(str).replace(' ', '',regex=True)+','
                            +rs_concat_2['CLABE'].astype(str).replace(' ', '',regex=True)+','
                            +rs_concat_2['amount'].astype(str).replace(' ', '',regex=True))
rs_concat_2.head()

In [ ]:
rs_concat_2.shape

# ARCHIVO DISPERSION

In [ ]:
disp1.head()

In [ ]:
disp1['report_date_local'] = pd.to_datetime(disp1['report_date_local'])

In [ ]:
split = disp1["swap"].str.split(",",n=7,expand=True)
split.head()

In [ ]:
disp1["swap"] = split[0] 
disp1["CLABE"] = split[3]

In [ ]:
disp1.head()

In [ ]:
disp1.loc[disp1['report_date_local'] == date_respuesta, 'tipo'] = 'Pago del día'
disp1.loc[disp1['report_date_local'] != date_respuesta, 'tipo'] = 'Reprogramación'
disp1.drop(['bank_name','CLABE','status'],axis=1,inplace=True)
disp1.head()

In [ ]:
disp_2 = disp1.merge(rs_concat_2[['report_id','status','Nombre de la institución','CLABE','LINEA CEP','Time_rs','Banco_emisor']]
                     ,on='report_id', how='left')
disp_2.sort_values(by=['status'])
disp_2["Time"].fillna('NO TIME',inplace=True)
disp_2.loc[disp_2['Time'] == 'NO TIME', 'status'] = 'ERROR'
disp_2.loc[(disp_2['LINEA CEP'].str.contains('nan')) & (['Nombre de la institución'] == 'BANAMEX'), 'status'] = 'ERROR'
# disp_2.loc[pd.notnull(disp_2['error_description']), 'status'] = 'ERROR'
disp_2.head()

In [ ]:
disp_2.loc[((disp_2['Banco_emisor'] == 'BANAMEX') &
            (disp_2['Nombre de la institución'] == 'BANAMEX')), 'LINEA CEP'] = 'COMPROBANTE MISMO BANCO BANAMEX'
disp_2.loc[((disp_2['Banco_emisor'] == 'SANTANDER') &
            (disp_2['Nombre de la institución'] == 'SANTANDER')), 'LINEA CEP'] = 'COMPROBANTE MISMO BANCO SANTANDER'
disp_2.loc[((disp_2['Banco_emisor'] == 'BANORTE') &
            (disp_2['Nombre de la institución'] == 'BANORTE')), 'LINEA CEP'] = 'COMPROBANTE MISMO BANCO BANORTE'
disp_2.loc[disp_2['status'] == 'ERROR', 'LINEA CEP'] = ''

In [ ]:
for col in disp_2.columns: 
    print(col) 

In [ ]:
disp_final = disp_2[['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','tipo'
                    ,'Nombre de la institución','swap','CLABE','status','LINEA CEP','Time','Banco_emisor']]
disp_final.rename(columns ={'Nombre de la institución':'bank','amount':'total_due','LINEA CEP':'CEP'}, inplace=True)
disp_final.sort_values(by=['status','Time'], ascending=True, inplace=True)
disp_final.drop_duplicates(subset =['report_id','Time'], keep ='first', inplace=True)
disp_final.head()

In [ ]:
disp_final.shape

# ERROR

In [ ]:
error = disp_final[(disp_final['status'] != 'PAID')]
error.head()

In [ ]:
error.shape

In [ ]:
error.drop(['swap','CLABE','status','CEP','Banco_emisor'],axis=1,inplace=True)

In [ ]:
error_final = error
error_final.head()

In [ ]:
error.loc[error['bank'] == 'BANAMEX', 'Check'] = 'SPEI DEVUELTO MISMO BANCO'
error.loc[error['bank'] != 'BANAMEX', 'Check'] = 'SPEI DEVUELTO'

In [ ]:
error.shape

In [ ]:
error_final = error.merge(rs_concat_2[['report_id','error_description']],on='report_id',how='left')
error_final.head()

In [ ]:
error_final.shape

# ARCHIVOS FINALES

### ARCHIVO DISPERSION

In [ ]:
writer = pd.ExcelWriter(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2022\07. JULIO\ARCHIVO DISPERSIONES FP\Dispersión-FP 2022-07-07.xlsx',
                        engine='xlsxwriter', engine_kwargs={'options': {'encoding':'utf-8'}})
disp_final.to_excel(writer, sheet_name='FAST PAYMENTS',index=False)
error_final.to_excel(writer, sheet_name='ERROR',index=False)
writer.close()

### ARCHIVO RESPUESTA  

In [ ]:
rs_concat.to_csv(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2022\07. JULIO\ARCHIVO RESPUESTAS FINALES\Respuesta-FP-ALL 2022-07-07.csv',index=False)

In [ ]:
# book = load_workbook('Dispersión-FP 2020-07-20.xlsx')
# writer = pd.ExcelWriter('Dispersión-FP 2020-07-20.xlsx', engine='openpyxl', options={'encoding':'utf-8'})
# writer.book = book
# match_final.to_excel(writer, sheet_name='FP-2 2020-07-20', index=False)
# writer.save()
# writer.close()